In [43]:
import pdb
import numpy as np
import tensorflow as tf
import pandas as pd
import h5py
import os
import matplotlib.pyplot as plt
import corner
import taurex.log
import csv # Per scrivere l'output su un .csv
import random

In [44]:
from IPython.display import Image, display
from FM_utils_final import setup_dedicated_fm
from tensorflow import keras
from tqdm import tqdm
from helper import *
from preprocessing import *
from submit_format import to_competition_format
from posterior_utils import *
from spectral_metric import *
from FM_utils_final import *
from MCDropout import MC_Convtrainer

In [45]:
taurex.log.disableLogging()

In [46]:
SEED=42 # Seed della run
RJUP = 69911000 # Raggio di Giove
MJUP = 1.898e27 # Massa di Giove
RSOL = 696340000 # Raggio del Sole
# Le mie directory sono all'interno del progetto
training_path = './Full_Dataset/Level2Data/'
test_path = './Full_Dataset/Level1Data/'
training_GT_path = os.path.join(training_path, 'Ground Truth Package') # Concatenamento del path
# Lettura dei file necessari
spectral_training_data = h5py.File(os.path.join(training_path,'SpectralData.hdf5'),"r")
aux_training_data = pd.read_csv(os.path.join(training_path,'AuxillaryTable.csv'))
soft_label_data = pd.read_csv(os.path.join(training_GT_path, 'FM_Parameter_Table.csv'))

Function helper

In [47]:
def visualise_spectrum(spectrum):
    fig = plt.figure(figsize=(10,6))
    ## multiple by 100 to turn it into percentage. 
    plt.errorbar(x=spectrum[:,0], y= spectrum[:,1]*100, yerr=spectrum[:,2]*100 )
    ## we tend to visualise it in log-scale
    plt.xscale('log')
    plt.xlabel('Wavelength (mircon)')
    plt.ylabel('Transit depth (%)')
    plt.show()

In [48]:
def plot_posterior(predicted_params,true_params,target_labels, base_fname=None):
    '''
    Plotta la distribuzione usando Corner
    Argomenti:
        predicted_params (np.ndarray) : Parametri predetti dal modello (shape: num_samples,num_parameters)
        true_params (np.ndarray) : Parametri 'reali' (di ARIEL)
        target_labels (list) : Lista dei parametri da usare per il plotting
    '''
    # Converti i parametri predetti e reali in array numpy (se non lo sono gia')
    predicted_params = np.array(predicted_params)
    true_params = np.array(true_params)

    ## DEBUG:
    print(f"Predicted parameters shape: {predicted_params.shape}")
    print(f"True parameters shape: {true_params.shape}")
    print(f"Target Labels: {target_labels}")

    # Ristruttura predicted_params se necessario
    if len(predicted_params.shape) == 1:
        predicted_params = predicted_params.reshape(-1, 1)

    # Filtra i valori NaN/inf
    if len(predicted_params.shape) == 2:
        valid_indices = np.all(np.isfinite(predicted_params), axis=1)
        filtered_pred_params = predicted_params[valid_indices]
    else:
        filtered_pred_params = predicted_params
    
    # Se non vi sono valori, esci
    if len(filtered_pred_params) == 0:
        print("Non vi sono valori validi")
        return
    # Crea il plot di Corner
    fig = corner.corner(
        filtered_pred_params,
        labels=target_labels,
        truths=true_params,
        show_titles=True
    )
    plt.suptitle("Distribuzione Bayesiana a Posteriori", fontsize=14) # Titolo
    if base_fname:
        directory = '/aggregate'
        full_path = get_unique_filename(base_fname,extension=".png",directory=directory)
        plt.savefig(full_path)
    plt.show()

In [49]:
def plot_single_parameter_distribution(predicted_params, true_params, parameter_index, parameter_name,base_filename=None):
    '''
    Plotta la distribuzione di un singolo parametro usando Matplotlib
    Argomenti:
        predicted_params (np.ndarray) : Parametri predetti dal modello (shape: num_samples,num_parameters)
        true_params (np.ndarray) : Parametri 'reali' (di ARIEL)
        parameter_index (int) : Indice del parametro da plottare
        parameter_name (str) : Nome del parametro da plottare
    '''
    # Converto i parametri predetti e reali in array NumPy
    predicted_params = np.array(predicted_params)
    true_params = np.array(true_params)
    # Verifica della forma
    if len(predicted_params.shape) == 1:
        predicted_params = predicted_params.reshape(-1,1)
    if len(true_params.shape) == 1:
        true_params = true_params.reshape(1,-1)
    # Estrai i dati del parametro specifico
    predicted_param = predicted_params[:, parameter_index]
    true_param = true_params[0, parameter_index]
    # Crea un istogramma per la distribuzione del parametro predetto
    plt.figure(figsize=(10, 6))
    plt.hist(predicted_param, bins=30, edgecolor='k', alpha=0.7, label='Predetto')
    plt.axvline(true_param, color='r', linestyle='dashed', linewidth=2, label='Reale')
    plt.title(f'Distribuzione Bayesiana del parametro: {parameter_name}')
    plt.xlabel(parameter_name) 
    plt.ylabel('Frequenza')
    plt.legend()
    plt.grid(True)
    if base_filename:
        directory = '/single_plots'
        full_path = get_unique_filename(parameter_name,extension=".png",directory=directory)
        plt.savefig(full_path)
    plt.show()
spec_matrix = to_observed_matrix(spectral_training_data,aux_training_data) # Definizione della matrice degli spettri
# print("spectral matrix shape:", spec_matrix.shape) # Stampa della 'shape' di spec_matrix

# Setting Valori del modello

In [ ]:
repeat = 5
threshold = 0.8 # Threshold per dati considerati 'validi'
N = 5000 # Svolgiamo il training solo sui primi 5000 dati
## extract the noise
noise = spec_matrix[:N,:,2]
## We will incorporate the noise profile into the observed spectrum by treating the noise as Gaussian noise.
spectra = spec_matrix[:N,:,1]
wl_channels = len(spec_matrix[0,:,0])
global_mean = np.mean(spectra)
global_std = np.std(spectra)
## add Rstar 
Rs = aux_training_data[['star_radius_m',]]
## we would prefer to use Rsol
Rs.loc[:,'star_radius'] = Rs['star_radius_m']/RSOL
Rs = Rs.drop(['star_radius_m'],axis=1)
Rs = Rs.iloc[:N, :]
mean_Rs = Rs.mean()
stdev_Rs = Rs.std()
target_labels = ['planet_ID','planet_temp','log_H2O','log_CO2','log_CO','log_CH4','log_NH3'] # Label
# print(soft_label_data.columns)
targets = soft_label_data.iloc[:N][target_labels] # Seleziono le righe puntate da "target_labels"
num_targets = targets.shape[1] # Estraggo il size dei target (il loro numero)
targets_mean = targets.mean() # Calcolo la media
targets_std = targets.std() # Standardizzazione

Sample dei primi 1000 valori di training

In [51]:
ind = np.zeros(len(spectra), dtype=bool)
ind[:1000] = True
#Inserisco i valori nel set appropriato
training_spectra, training_Rs,training_targets, training_noise = spectra[ind],Rs[ind],targets[ind], noise[ind] 
valid_spectra, valid_Rs, valid_targets = spectra[~ind],Rs[~ind],targets[~ind] 
aug_spectra = augment_data_with_noise(training_spectra, training_noise, repeat) # Aumento tramite rumore (Applico il rumore al training set)
aug_Rs = np.tile(training_Rs.values,(repeat,1)) # Duplico le righe in un array numpy, rimuovendo le etichette di riga e colonna
aug_targets = np.tile(training_targets.values,(repeat,1)) # Eseguo la stessa operazione per i targets
# SPETTRI
std_aug_spectra = standardise(aug_spectra, global_mean, global_std)
std_aug_spectra = std_aug_spectra.reshape(-1, wl_channels)
std_valid_spectra = standardise(valid_spectra, global_mean, global_std)
std_valid_spectra = std_valid_spectra.reshape(-1, wl_channels)

RAGGI

In [52]:
std_aug_Rs= standardise(aug_Rs, mean_Rs.values.reshape(1,-1), stdev_Rs.values.reshape(1,-1))
std_valid_Rs= standardise(valid_Rs, mean_Rs, stdev_Rs)

TARGETS

In [53]:
std_aug_targets = standardise(aug_targets, targets_mean.values.reshape(1,-1), targets_std.values.reshape(1,-1))
std_valid_targets = standardise(valid_targets, targets_mean, targets_std)
batch_size= 32
lr= 1e-3
epochs = 30
filters = [32,64,64]
dropout = 0.1
# number of examples to generate in evaluation time (5000 is max for this competition)
N_samples = 5000
model = MC_Convtrainer(wl_channels,num_targets,dropout,filters) # Seguendo Yip et al. [https://iopscience.iop.org/article/10.3847/1538-3881/ac1744%3E]
model.compile(
    optimizer=keras.optimizers.Adam(lr),
    loss='mse',
)
# DATA TRAINING : SPECIFICI SAMPLE
# Tutto l'array
train_indices = np.arange(len(std_valid_spectra))

Seleziono i dati dal 1o al 1000o<br>
train_indices = np.arange(0,1000)

Seleziono i dati dal 1000o al 2000o<br>
train_indices = np.arange(1000,2000)

Seleziono i dati dal 2000o al 3000o<br>
train_indices = np.arange(2000,3000)

Seleziono i dati dal 4000o al 5000o<br>
train_indices = np.arange(4000,5000)

In [54]:
train_spectra = std_aug_spectra[train_indices]
train_Rs = std_aug_Rs[train_indices]
train_targets = std_aug_targets[train_indices]

Train il modello solo sui dati selezionati

In [ ]:
model.fit(
    [train_spectra, train_Rs], 
    train_targets, 
    validation_data=([std_valid_spectra, std_valid_Rs], std_valid_targets),
    batch_size=batch_size, 
    epochs=epochs, 
    shuffle=False,
)

Selezionamento del Ground Truth dei dati di validazione, nell'ordine corretto

In [ ]:
index= np.arange(len(ind))
valid_index = index[~ind]
print(f"valid_idx: {valid_index}")
instances = N_samples
y_valid_distribution = np.zeros((instances, len(std_valid_spectra), num_targets )) # Creo un array di zeri

Effettuo *instances* previsioni<br>
for i in tqdm(range(instances)): # OLD (REMOVED)

In [ ]:
# Effettuo *instances* previsioni
y_valid_distribution_np = np.zeros((instances, len(std_valid_spectra), num_targets)) # Initialize a NumPy array
for i in tqdm(range(instances)):
    y_pred_valid = model([std_valid_spectra, std_valid_Rs], training=True)
    y_valid_distribution_np[i] = np.array(y_pred_valid) # Assign values to the NumPy array

# Rimodellamento dei dati
y_valid_distribution_np = y_valid_distribution_np.reshape(-1, num_targets)
# Denormalizzazione delle previsioni, e rimodellamento dell'array
y_pred_valid_org = transform_and_reshape(y_valid_distribution_np, global_mean, global_std, instances, N_testdata=len(std_valid_spectra))

tr1 = y_pred_valid_org
# I pesi prendono in considerazione l'importanza di ogni punto nei dati delle traccie
weights1 = np.ones((tr1.shape[0], tr1.shape[1])) / tr1.shape[1]

Loading del Ground Truth delle tracce

In [ ]:
trace_GT = h5py.File(os.path.join(training_GT_path, 'TraceData.hdf5'),"r")
'''
posterior_scores = []
bounds_matrix = default_prior_bounds()
for idx, pl_idx in enumerate(valid_index):
    tr_GT = trace_GT[f'Planet_train{pl_idx+1}']['tracedata'][()]
    weights_GT = trace_GT[f'Planet_train{pl_idx+1}']['weights'][()]
    ## there are cases without ground truth, we will skip over them for this baseline
    ## but every example in leaderboard and final evaluation set will have a complementary ground truth
    if np.isnan(tr_GT).sum() == 1:
        continue
    # compute posterior loss
    score = compute_posterior_loss(tr1[idx], weights1[idx], tr_GT, weights_GT, bounds_matrix)
    posterior_scores.append(score)
avg_posterior_score = np.mean(posterior_scores)
print(avg_posterior_score)
'''

In [ ]:
import csv # Per scrivere l'output su un .csv
import random

In [ ]:
from IPython.display import Image, display

In [ ]:
N_examples = 10 ## number of test examples to go through
N_samples = 10 ## number of quantiles to sample (fixed to 10 in the competition)
q_list = np.linspace(0.01,0.99,N_samples)
## beta - weight of the posterior loss [0,1], and the weight of spectral loss will decrease accordingly. 
beta = 0.8

Setting dei path per XSEC (opacitÃ ) e CIA (Collision Induced Absorption)

In [ ]:
opacity_path=os.path.join(os.getcwd(), 'XSEC')
CIA_path=os.path.join(os.getcwd(), 'HITRAN')

Leggi la Spectral Grid

In [ ]:
ariel_wlgrid, ariel_wlwidth, ariel_wngrid, ariel_wnwidth = ariel_resolution()

Inizializza il modello T3 di base per ADC2023

In [ ]:
fm = initialise_forward_model(opacity_path, CIA_path)

Lettura delle info ausiliare

In [ ]:
aux_df = aux_training_data
# Conferma della corrispondenza delle dimensioni
Rs = aux_df['star_radius_m']/RSOL
# Rp = aux_df['planet_radius_m']/RJUP
Mp = aux_df['planet_mass_kg']/MJUP

!!!!!!!!!!!!!!!!!!!

In [ ]:
spectral_scores = []
bounds_matrix = default_prior_bounds()
output_file = open('spectral_scores.csv', mode='w', newline='\n')
writer = csv.writer(output_file)
writer.writerow(['Planet Index', 'Spectral Score'])

Scelgo casualmente 20 elementi<br>
 Test : Selezioniamo 20 elementi per volta, vediamo i valori risultanti   

In [ ]:
start_idx = random.randint(0,len(valid_index)-20)
end_idx = start_idx+20

La slice di valid index conterra' esattamente 20 elementi

In [ ]:
valid_index_slice = valid_index[start_idx:end_idx]

print(f"Slice di indici: {valid_index_slice}")

In [ ]:
predicted_params = []
true_params = []
for i,pl_idx in enumerate(valid_index_slice):# Effettuiamo solo 20 iterazioni
    # print(f"pl_idx : {pl_idx}")
    
    idx = 0
    tr_GT = trace_GT[f'Planet_train{pl_idx+1}']['tracedata'][()]
    weights_GT = trace_GT[f'Planet_train{pl_idx+1}']['weights'][()]
    
    # Settiamo un limite inferiore ai pesi
    min_w = 1e-8 # Come il bound
    
    proxy_compute_spectrum = setup_dedicated_fm(fm, idx, Rs, Mp, ariel_wngrid, ariel_wnwidth)
    weights_GT = weights_GT/np.sum(weights_GT) # Normalizzazione dei pesi
    
    # check_output(tr_GT,weights_GT,proxy_compute_spectrum)
    if np.isnan(tr_GT).sum()>0 or np.isinf(tr_GT).sum()>0:
        print(f"Not-a-Number or INF found for planet {pl_idx+1}")
        idx += 1
        continue
    if np.isnan(weights_GT).sum()>0 or np.isinf(weights_GT).sum()>0:
        print(f"Not-a-Number or INF found for planet {pl_idx+1}")                               
        idx += 1
        continue
    try:
        score = compute_spectral_loss(tr1[i], weights1[i], tr_GT,weights_GT,bounds_matrix,proxy_compute_spectrum,q_list) # Attenzione al problema della versione di NumPy (1.26) per taurex e tensorflow
    except (IndexError, FloatingPointError) as e:
        print(f"Error: {e} for planet {pl_idx+1}")
        idx += 1
        continue
    if not np.isnan(score):
        spectral_scores.append(score)
        writer.writerow([pl_idx+1,score]) # Scrivi l'oputput
    idx += 1
    # Calcolo la predizione ed i valori reali
    pl_predicted_params = y_pred_valid_org[i*N_samples : (i+1)*N_samples]
    pl_true_params = valid_targets.iloc[i].values # i rappresenta l'indice di valid_index_slice, volendo usare la i-sima riga di 'valid_targets'
    # Aggiungo la predizione e il valore vero alle liste di tutti i valori
    predicted_params.append(pl_predicted_params)
    true_params.append(pl_true_params)

In [ ]:
predicted_params = np.concatenate(predicted_params,axis=0)
true_params = np.mean(true_params,axis=0)

Calcolo del punteggio medio

In [ ]:
avg_spectral_score = np.mean(spectral_scores)
print(f"avg_spectral_score: {avg_spectral_score}")
# Image('path_length_layer_0.png')
output_file.close()
# !!!!!!!!!!!!!!!!!!!

In [ ]:
final_score = (1-beta)*avg_spectral_score # Removed posterior loss, since we haven't yet verified it
print(f"final loss is {final_score:.4f}")
spec_test_data = h5py.File(os.path.join(test_path,'SpectralData.hdf5'),"r")
aux_test_data = pd.read_csv(os.path.join(test_path,'AuxillaryTable.csv'))
test_spec_matrix = to_observed_matrix(spec_test_data,aux_test_data )
std_test_spectra = standardise(test_spec_matrix[:,:,1], global_mean, global_std)
test_Rs = aux_test_data[['star_radius_m']]
# Preferiamo usare RSOL
test_Rs.loc[:,'star_radius'] = test_Rs['star_radius_m']/RSOL
test_Rs = test_Rs.drop(['star_radius_m'],axis=1)
std_test_Rs= standardise(test_Rs, mean_Rs, stdev_Rs)
tf.keras.utils.set_random_seed(SEED) # Importazione del SEED
instances = N_samples
y_pred_distribution = np.zeros((instances, len(std_test_spectra), num_targets ))
for i in tqdm(range(instances)):
    y_pred = model([std_test_spectra,std_test_Rs],training=True)
    y_pred_distribution[i] += y_pred
y_pred_distribution = y_pred_distribution.reshape(-1,num_targets)
y_pred_test_org = transform_and_reshape(y_pred_distribution,targets_mean, targets_std,instances,N_testdata=len(std_test_spectra))
tracedata = y_pred_test_org
# I pesi prendono in considerazione l'importanza di ciascun dati
weight = np.ones((tracedata.shape[0],tracedata.shape[1]))/np.sum(np.ones(tracedata.shape[1]) )
submission = to_competition_format(tracedata, weight, name="submission.hdf5")
# import matplotlib as plt
file_path = './spectral_scores.csv'
df = pd.read_csv(file_path)

Prime righe

In [ ]:
print(df.head())

Calcolo delle statistiche descrittive

In [ ]:
mean_score = df['Spectral Score'].mean()
median_score = df['Spectral Score'].median()
min_score = df['Spectral Score'].min()
max_score = df['Spectral Score'].max()
std_score = df['Spectral Score'].std()

Trova l'indice del massimo score

In [ ]:
if(len(df) > 0):
    max_score_idx = df['Spectral Score'].idxmax()   
    if(max_score_idx <= len(df['Spectral Score'])):
        # Stampa i parametri del pianeta con il massimo score
        print(f"Parametri del pianeta con massimo score (indice {max_score_idx}):")
        print(valid_targets.iloc[max_score_idx])
    # Visualizzazione dei punteggi
    plt.figure(figsize=(10,6))
    plt.hist(df['Spectral Score'],bins=20,edgecolor='k',alpha=0.7)
    plt.title('Distribuzione Spectral Scores')
    plt.xlabel('Spectral Score')
    plt.ylabel('Frequenza')
    plt.grid(True)
    plt.show()
else:
    print('Empty sequence')

In [ ]:
plot_posterior(predicted_params,true_params,target_labels,base_fname="aggregate_posterior")

 Plotting di ogni feature<br>
filename = "plot_R.png"

In [ ]:
plot_single_parameter_distribution(predicted_params,true_params,0,"planet_radius","plot_R")
# filename = "plot_T.png"
plot_single_parameter_distribution(predicted_params,true_params,1,"planet_temp","plot_T")
# filename = "plot_H2O.png"
plot_single_parameter_distribution(predicted_params,true_params,2,"log_H2O","Plot_H2O")
# filename = "plot_CO2.png"
plot_single_parameter_distribution(predicted_params,true_params,3,"log_CO2","Plot_H2O")
# filename = "plot_CO.png"
plot_single_parameter_distribution(predicted_params,true_params,4,"log_CO","Plot_H2O")
# filename = "plot_CH4.png"
plot_single_parameter_distribution(predicted_params,true_params,5,"log_CH4","Plot_H2O")
# filename = "plot_NH3.png"
plot_single_parameter_distribution(predicted_params,true_params,6,"log_NH3","Plot_H2O")